# Data Evaluation for Multi-Grasp motion planner

In [3]:
# Common imports
import pandas as pd
import numpy as np
import os

## Profiling
This section contains code to load, average and visualize profiling results.

In [110]:
    def load_stats(filename):
    """Load profiling stats for an algorithm from the given file.
    Args:
        filename, string: name of the stats file.
    Returns:
        pd.DataFrame: Profiling stats with standard deviations
    """
    run_stats = pd.read_csv(filename)
    avg_run_stats = run_stats.groupby('function_name').mean()
    time_columns = ['total_runtime', 'avg_runtime', 'total_cpu_time', 'avg_cpu_time']
    time_columns_std = [name + "_std" for name in time_columns]
    avg_run_stats[time_columns_std] = run_stats.groupby('function_name').std()[time_columns]
    return avg_run_stats

Show the profiling stats of a specific algorithm:

In [15]:
load_stats('/tmp/test_results/LWAstar__MultiGraspGraph__Standard2DScene__2/run_stats')

,num_calls,total_runtime,avg_runtime,total_cpu_time,avg_cpu_time,total_runtime_std,avg_runtime_std,total_cpu_time_std,avg_cpu_time_std
function_name,,,,,,,,,
IntegralEdgeCostComputer::lowerBound,8668,0.004136,4.771902e-07,0.004714,5.438970e-07,2.085157e-04,2.405572e-08,1.841981e-04,2.125046e-08
MultiGraspRoadmapGraph::checkValidity,1698,0.149892,8.827579e-05,0.149241,8.789240e-05,4.261395e-02,2.509644e-05,4.256103e-02,2.506538e-05
MultiGraspRoadmapGraph::getEdgeCost,16870,0.185707,1.100810e-05,0.186170,1.103559e-05,3.547900e-03,2.103310e-07,3.722260e-03,2.206314e-07
MultiGraspRoadmapGraph::getGoalCost,1,0.000002,2.420900e-06,0.000003,2.700000e-06,3.116716e-07,3.116716e-07,4.830459e-07,4.830459e-07
MultiGraspRoadmapGraph::getSuccessors,417,0.018291,4.386270e-05,0.018276,4.382782e-05,4.415900e-04,1.058986e-06,4.227946e-04,1.013901e-06
MultiGraspRoadmapGraph::heuristic,10421,0.007566,7.260091e-07,0.008333,7.995970e-07,1.521634e-04,1.460176e-08,1.777465e-04,1.705646e-08
MultiGraspRoadmapGraph::isGoal,418,0.000236,5.651833e-07,0.000278,6.657896e-07,5.506353e-06,1.317290e-08,1.016585e-05,2.432026e-08
Roadmap::computeCostWithGrasp,451,0.159262,3.531293e-04,0.158733,3.519580e-04,3.017736e-03,6.691171e-06,3.113742e-03,6.904119e-06
Roadmap::densify,1,0.037720,3.772026e-02,0.037429,3.742950e-02,1.938049e-03,1.938049e-03,1.729018e-03,1.729018e-03


## Loading summarized results
This section contains code to load summarized results, i.e. number of evaluated nodes, runtime, number of evaluated edges, ...

In [13]:
algo_to_function = {
    "LazySP_LWLPAstar": 'lazySP',
    "LWAstar": "lwaStarSearch"
    # TODO add other algorithm to function mappings here
}

def count_event(log_line, storage):
    """ Count a roadmap evalation log event.
    Args:
        log_line, str - a line from a log file.
        storage, dict - dictionary to update counts in.
            Needs to have keys 'base_validity', 'grasp_validity', 'base_cost', 'grasp_cost', 'base_expansions'. 'grasp_expansions'
    """
    line_args = log_line.split(',')
    storage['base_validity'] += line_args[0] == "VAL_BASE"
    storage['grasp_validity'] += line_args[0] == "VAL_GRASP"
    storage['base_cost'] += line_args[0] == "EDGE_COST"
    storage['grasp_cost'] += line_args[0] == "EDGE_COST_GRASP"
    storage['base_expansions'] += line_args[0] == "BASE_EXPANSION"
    storage['grasp_expansions'] += line_args[0] == "EXPANSION"

def load_test_case_result(result_folder, runtime_type='total_runtime'):
    """Load the result of a single test case and return it as a pandas.Series.
    Args:
        result_folder, string: The folder containing the results of the test case.
            The folder should contain three files 'log_evaluation', 'log_roadmap', 'run_stats' produced
            by testing the MGSearchMotionPlanner. The name of the result folder should follow the format
            <algo_name>__<graph_name>__<scene_name>__<num_grasps>
    """
    folder_name = os.path.basename(os.path.normpath(result_folder))
    algo_name, graph_name, scene_name, num_grasps = folder_name.split('__')
    num_grasps = int(num_grasps)
    # load runtime statistics
    avg_run_stats = load_stats(result_folder + '/run_stats')
    # read number of roadmap nodes
    with open(result_folder + '/log_roadmap', 'r') as roadmap_file:
        num_nodes = len(roadmap_file.readlines())
    # read number of validity checks, edge evaluations and expansions
    roadmap_evaluations = {'base_validity': 0, 'grasp_validity': 0, 'base_cost': 0, 'grasp_cost': 0, 'base_expansions': 0, 'grasp_expansions': 0}
    with open(result_folder + '/log_evaluation', 'r') as evalulation_file:
        for line in evalulation_file:
            count_event(line, roadmap_evaluations)
    # Create Series for this test case
    return pd.Series({'algorithm': algo_name, 
                      'graph': graph_name, 
                      'runtime': avg_run_stats.loc[algo_to_function[algo_name], runtime_type],
                      'runtime_std': avg_run_stats.loc[algo_to_function[algo_name], runtime_type + '_std'], 
                      'base_validity_checks': roadmap_evaluations['base_validity'],
                      'grasp_validity_checks': roadmap_evaluations['grasp_validity'],
                      'base_edge_cost': roadmap_evaluations['base_cost'],
                      'grasp_edge_cost': roadmap_evaluations['grasp_cost'],
                      'base_expansions': roadmap_evaluations['base_expansions'],
                      'grasp_expansions': roadmap_evaluations['grasp_expansions'],
                      'roadmap_size': num_nodes,
                      'scene': scene_name, 
                      'num_grasps': num_grasps})

def load_results(results_base_folder):
    """Load all results in the given base folder and return in a single DataFrame.
    Args:
        results_base_folder, str: Path to the folder containing various result sub-folders.
    Returns:
        pd.DataFrame: Data frame containing all results.
    """
    series = []
    for fname in os.listdir(results_base_folder):
        series.append(load_test_case_result(results_base_folder + '/' + fname))
    return pd.DataFrame(series)

In [16]:
# read all results in a folder
load_results('/tmp/test_results')


,algorithm,graph,runtime,runtime_std,base_validity_checks,grasp_validity_checks,base_edge_cost,grasp_edge_cost,base_expansions,grasp_expansions,roadmap_size,scene,num_grasps
0,LWAstar,MultiGraspGraph,0.380269,0.042530,505,610,0,451,1,416,10006,Standard2DScene,2
1,LWAstar,MultiGraspGraph,0.206689,0.003217,102,103,0,12,1,3,1101,YumiTestScene,6
2,LazySP_LWLPAstar,LazyWeightedMultiGraspGraph,4.402683,0.071554,584,80,555,64,1,33781,10006,Standard2DScene,2
3,LazySP_LWLPAstar,LazyWeightedMultiGraspGraph,0.258329,0.008403,102,102,12,2,1,15,1101,YumiTestScene,6


## Plotting results
This section defines some functions to plot results.